In [1]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

## Preprocess Housing Price Data

In [2]:
housing_df = pd.read_csv('data/nsw_suburb_median_price -2007-2020.csv')
housing_df

,date,property_type,median_price,sales,suburb,postcode,state
0,31/03/2008,house,587406,1,ABBOTSBURY,2176,NSW
1,30/06/2008,house,587406,2,ABBOTSBURY,2176,NSW
2,30/09/2008,house,587406,5,ABBOTSBURY,2176,NSW
3,31/12/2008,house,587406,4,ABBOTSBURY,2176,NSW
4,31/03/2009,house,590711,2,ABBOTSBURY,2176,NSW
...,...,...,...,...,...,...,...
69224,31/03/2019,unit,816049,8,ZETLAND,2017,NSW
69225,30/06/2019,unit,811758,11,ZETLAND,2017,NSW
69226,30/09/2019,unit,808139,11,ZETLAND,2017,NSW
69227,31/12/2019,unit,804726,19,ZETLAND,2017,NSW


In [3]:
# Convert suburb names from UPPERCASE to Title Case
housing_df['suburb'] = housing_df['suburb'].str.title()
# Replace 'Mount Kuring-Gai' with 'Mount Kuring-gai' for consistency with the suburb list
housing_df['suburb'] = housing_df['suburb'].replace('Mount Kuring-Gai', 'Mount Kuring-gai')
# Drop irrelevant columns
housing_df.drop(columns={'sales', 'postcode', 'state'}, inplace=True)
# Convert date values into a machine-readable format
housing_df['date'] = pd.to_datetime(housing_df['date'], format='%d/%m/%Y')
housing_df

,date,property_type,median_price,suburb
0,2008-03-31,house,587406,Abbotsbury
1,2008-06-30,house,587406,Abbotsbury
2,2008-09-30,house,587406,Abbotsbury
3,2008-12-31,house,587406,Abbotsbury
4,2009-03-31,house,590711,Abbotsbury
...,...,...,...,...
69224,2019-03-31,unit,816049,Zetland
69225,2019-06-30,unit,811758,Zetland
69226,2019-09-30,unit,808139,Zetland
69227,2019-12-31,unit,804726,Zetland


## Explore Data Consistency

In [4]:
len(housing_df['suburb'].unique())

1157

In [5]:
# Check average date of sales
housing_df.groupby(by=['suburb'])

In [6]:
housing_df['date'][1] - housing_df['date'][0]

Timedelta('91 days 00:00:00')

## Calculate Average Housing Prices

In [7]:
house_price = housing_df[housing_df['property_type'] == 'house'].groupby(by=['suburb']).mean()
house_price.rename(columns={'median_price': 'house_price'}, inplace=True)
house_price

,house_price
suburb,
Abbotsbury,8.313419e+05
Abbotsford,1.775096e+06
Aberdare,2.707604e+05
Aberdeen,2.892939e+05
Aberglasslyn,4.085584e+05
...,...
Yarrawarrah,8.103405e+05
Yass,4.307834e+05
Yerrinbool,4.647730e+05


In [8]:
unit_price = housing_df[housing_df['property_type'] == 'unit'].groupby(by=['suburb']).mean()
unit_price.rename(columns={'median_price': 'unit_price'}, inplace=True)
unit_price

,unit_price
suburb,
Abbotsford,1.007256e+06
Adamstown,3.245622e+05
Albury,2.378140e+05
Alexandria,5.962114e+05
Allawah,5.575098e+05
...,...
Woolooware,4.475582e+05
Woonona,3.731087e+05
Yagoona,3.774474e+05


## Extract Data of Concerned Suburbs 

The housing dataset contains data of 1157 unique suburbs. Therefore, I will exclude irrelevant data and only maintain that of 152 identified suburbs with a train station.

In [9]:
# Load the final suburb list
suburb_df = pd.read_csv('data/sydney_suburbs.csv')
suburb_df

,suburb,lat,lon
0,Allawah,-33.970018,151.114517
1,Arncliffe,-33.936592,151.146805
2,Artarmon,-33.808087,151.192733
3,Ashfield,-34.096505,150.778939
4,Asquith,-33.687484,151.108685
...,...,...,...
147,Wolli Creek,-33.930744,151.155272
148,Wollstonecraft,-33.828158,151.196621
149,Woolooware,-34.048276,151.141431
150,Yagoona,-33.907725,151.026108


In [10]:
# Merge suburb dataframe with housing dataframe
df = suburb_df.merge(house_price, on='suburb', how='left').merge(unit_price, on='suburb', how='left')
df

,suburb,lat,lon,house_price,unit_price
0,Allawah,-33.970018,151.114517,NaN,557509.816327
1,Arncliffe,-33.936592,151.146805,9.298207e+05,526500.617021
2,Artarmon,-33.808087,151.192733,1.908052e+06,598473.632653
3,Ashfield,-34.096505,150.778939,1.118246e+06,551060.122449
4,Asquith,-33.687484,151.108685,9.041630e+05,670753.857143
...,...,...,...,...,...
147,Wolli Creek,-33.930744,151.155272,NaN,547217.775510
148,Wollstonecraft,-33.828158,151.196621,2.218654e+06,809956.673469
149,Woolooware,-34.048276,151.141431,1.213125e+06,447558.215686
150,Yagoona,-33.907725,151.026108,6.452034e+05,377447.404255


In [27]:
# List of suburb with missing housing records 
suburb_missing = df[(df['house_price'].isna() & df['unit_price'].notna()) | 
                    (df['house_price'].notna() & df['unit_price'].isna()) | 
                    (df['house_price'].isna() & df['unit_price'].isna())]
print(f'Total number of suburbs with missing housing records: {len(suburb_missing.index)}')

Total number of suburbs with missing housing records: 60


In [28]:
housing_df['property_type'].value_counts()

house    54147
unit     15082
Name: property_type, dtype: int64